In [161]:
from llama_parse import LlamaParse
from dotenv import load_dotenv
import os

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

QDRANT_API = os.getenv("QDRANT_API")
QDRANT_URL = os.getenv("QDRANT_URL")

LLAMA_PARSE_API = os.getenv("LLAMA_PARSE_API")

VOYAGE_API = os.getenv("VOYAGE_API")

In [162]:
# file_path ='data/Kazakhstan_tarihi_7_atamura_sample.pdf'

DISCIPLINE = "Қазақстан тарихы"
GRADE = "10"
PUBLISHER = "Мектеп"

collection_name = "Docker-Redis_test_sample_tests_4"

In [ ]:
import uuid
import re
from llama_parse import LlamaParse
from qdrant_client import QdrantClient, models
from langchain_text_splitters import MarkdownHeaderTextSplitter


from FlagEmbedding import BGEM3FlagModel
from langchain_voyageai import VoyageAIEmbeddings



# Initialize Voyage Large
dense_model = VoyageAIEmbeddings(
    voyage_api_key=VOYAGE_API, 
    model="voyage-3-large",
    output_dimension=1024
)
sparse_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API)
collection_name = "Kazakh_History_Semantic_Chunks1024_2"

# 1. Создание коллекции (Hybrid)
if not client.collection_exists(collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config={
            "voyage-dense": models.VectorParams(
                size=1024, # Размер для Gemini
                distance=models.Distance.COSINE
            )
        },
        sparse_vectors_config={
            "bge-sparse": models.SparseVectorParams(
                index=models.SparseIndexParams(on_disk=True)
            )
        }
    )


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 21337.82it/s]


In [ ]:
file_path="Kazakhstan_tarihi_7_atamura_Extracted3.pdf"

# 2. Парсинг и семантическое разделение

parser = LlamaParse(
    api_key=LLAMA_PARSE_API,
    parse_mode="parse_page_with_lvm",
    model="gemini-2.5-pro",
    vendor_multimodal_api_key=GEMINI_API_KEY,
    result_type="markdown",
    high_res_ocr=True,
    adaptive_long_table=True,  # Adaptive long table. LlamaParse will try to detect long table and adapt the output
    outlined_table_extraction=True,  # Whether to try to extract outlined tables
)

print("Парсинг через LlamaParse...")
json_results = parser.get_json_result(file_path)
pages = json_results[0].get("pages", [])

# Собираем всю книгу в один текст и сохраняем базу картинок
full_markdown = ""
image_vault = {} # {filename: base64}

for page in pages:
    full_markdown += page.get("md", "") + "\n"
    for img in page.get("images", []):
        if img.get("name") and img.get("base64"):
            image_vault[img.get("name")] = img.get("base64")


Парсинг через LlamaParse...
Started parsing the file under job_id d0dc7402-4ff8-4281-9ab5-53d3a2291563
..

In [164]:

# Настраиваем разделитель по заголовкам Markdown
headers_to_split_on = [
    ("#", "Header_1"),
    ("##", "Header_2"),
    ("###", "Header_3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, 
    strip_headers=False # Оставляем заголовки внутри текста для векторизации
)

# Режем текст на логические разделы (чанки)
semantic_chunks = markdown_splitter.split_text(full_markdown)
semantic_chunks

[Document(metadata={'Header_1': 'БІРІНШІ БӨЛІМ ҚАЗАҚ-ЖОҢҒАР СОҒЫСТАРЫ', 'Header_2': '§1-2. ЖОҢҒАР БАСҚЫНШЫЛАРЫНА ҚАРСЫ ОТАН СОҒЫСЫНЫҢ БАСТАЛУЫ'}, page_content='# БІРІНШІ БӨЛІМ ҚАЗАҚ-ЖОҢҒАР СОҒЫСТАРЫ  \n## §1-2. ЖОҢҒАР БАСҚЫНШЫЛАРЫНА ҚАРСЫ ОТАН СОҒЫСЫНЫҢ БАСТАЛУЫ  \n> Бүгінгі сабағымызда Жоңғар мемлекетінің құрылуы мен басқыншылық жорықтарының себептерін талқылаймыз. Жоңғар шапқыншылығының зардаптарын, демографиялық өзгерістерді анықтаймыз. Халық ауыз әдебиеті мен музыка туындыларының тарихи дереккөз ретіндегі құндылығын бағалаймыз.  \nҚазақтар жоңғарлармен ұзақ уақыт бойы кескілескен қанды шайқастар жүргізіп келді. Алайда соның ішіндегі ең ауыры «Ақтабан шұбырынды, Алқакөл сұлама» атанған, қайғы мен қасіретке толы апатты кезең болды.'),
 Document(metadata={'Header_1': 'БІРІНШІ БӨЛІМ ҚАЗАҚ-ЖОҢҒАР СОҒЫСТАРЫ', 'Header_2': '§1-2. ЖОҢҒАР БАСҚЫНШЫЛАРЫНА ҚАРСЫ ОТАН СОҒЫСЫНЫҢ БАСТАЛУЫ', 'Header_3': '1. Қазақ хандығының XVIII ғасырдың бас кезіндегі ішкі және сыртқы жағдайы.'}, page_content='###

In [154]:
with open("full_markdown.md", "w", encoding="utf-8") as f:
    f.write(full_markdown)

In [111]:
semantic_chunks

[Document(metadata={'Header_1': 'БІРІНШІ БӨЛІМ ҚАЗАҚ-ЖОҢҒАР СОҒЫСТАРЫ', 'Header_2': '§1-2. ЖОҢҒАР БАСҚЫНШЫЛАРЫНА ҚАРСЫ ОТАН СОҒЫСЫНЫҢ БАСТАЛУЫ'}, page_content='# БІРІНШІ БӨЛІМ ҚАЗАҚ-ЖОҢҒАР СОҒЫСТАРЫ  \n## §1-2. ЖОҢҒАР БАСҚЫНШЫЛАРЫНА ҚАРСЫ ОТАН СОҒЫСЫНЫҢ БАСТАЛУЫ  \n> Бүгінгі сабағымызда Жоңғар мемлекетінің құрылуы мен басқыншылық жорықтарының себептерін талқылаймыз. Жоңғар шапқыншылығының зардаптарын, демографиялық өзгерістерді анықтаймыз. Халық ауыз әдебиеті мен музыка туындыларының тарихи дереккөз ретіндегі құндылығын бағалаймыз.  \nҚазақтар жоңғарлармен ұзақ уақыт бойы кескілескен қанды шайқастар жүргізіп келді. Алайда соның ішіндегі ең ауыры «Ақтабан шұбырынды, Алқакөл сұлама» атанған, қайғы мен қасіретке толы апатты кезең болды.'),
 Document(metadata={'Header_1': 'БІРІНШІ БӨЛІМ ҚАЗАҚ-ЖОҢҒАР СОҒЫСТАРЫ', 'Header_2': '§1-2. ЖОҢҒАР БАСҚЫНШЫЛАРЫНА ҚАРСЫ ОТАН СОҒЫСЫНЫҢ БАСТАЛУЫ', 'Header_3': '1. Қазақ хандығының XVIII ғасырдың бас кезіндегі ішкі және сыртқы жағдайы.'}, page_content='###

In [165]:
points = []
print(f"Создано {len(semantic_chunks)} логических разделов. Начинаем эмбеддинг...")

text_documents = []
chunk_images_batchs = []

for chunk in semantic_chunks:
    text_content = chunk.page_content
    if not text_content.strip():
        continue

    # Ищем, какие картинки упоминаются в этом чанке
    # LlamaParse вставляет их как ![](image_name.png)
    found_images_names = re.findall(r"!\[\]\((.*?)\)", text_content)
    chunk_images_b64 = [image_vault[name] for name in found_images_names if name in image_vault]

    text_documents.append(text_content)
    chunk_images_batchs.append(chunk_images_b64)

text_documents

Создано 5 логических разделов. Начинаем эмбеддинг...


['# БІРІНШІ БӨЛІМ ҚАЗАҚ-ЖОҢҒАР СОҒЫСТАРЫ  \n## §1-2. ЖОҢҒАР БАСҚЫНШЫЛАРЫНА ҚАРСЫ ОТАН СОҒЫСЫНЫҢ БАСТАЛУЫ  \n> Бүгінгі сабағымызда Жоңғар мемлекетінің құрылуы мен басқыншылық жорықтарының себептерін талқылаймыз. Жоңғар шапқыншылығының зардаптарын, демографиялық өзгерістерді анықтаймыз. Халық ауыз әдебиеті мен музыка туындыларының тарихи дереккөз ретіндегі құндылығын бағалаймыз.  \nҚазақтар жоңғарлармен ұзақ уақыт бойы кескілескен қанды шайқастар жүргізіп келді. Алайда соның ішіндегі ең ауыры «Ақтабан шұбырынды, Алқакөл сұлама» атанған, қайғы мен қасіретке толы апатты кезең болды.',
 '### 1. Қазақ хандығының XVIII ғасырдың бас кезіндегі ішкі және сыртқы жағдайы.  \n**Тірек сөздер:**\n* Жоңғар\n* Қайып хан\n* Канси\n* Сепаратизм\n* Цеван-Рабтан\n* «Елім-ай»  \n**Анықта!**\nҚазақ қоғамында сепаратизмнің күшеюіне не әсер етті?  \nТәуке хан билік еткен жылдары Қазақ хандығы орталықтандырылған мемлекет болып қала берді. Қазақ хандығында саяси тұрақтылық орнады, ішкі алауыздық азайды. Ауызбірш

In [166]:
def sparse_documents(corpus):
    output = sparse_model.encode(corpus,
    return_dense=False, 
    return_sparse=True, 
    return_colbert_vecs=False)

    batch_keys = []
    batch_values = []
    for batch in output["lexical_weights"]:
        batch_keys.append([int(k) for k in batch.keys()])
        batch_values.append([float(v) for v in batch.values()])
    return batch_keys, batch_values
    

# Генерация векторов (Dense + Sparse)
print("Генерация векторов...")
dense_vectors = dense_model.embed_documents(text_documents)

print("Генерация разреженных векторов...")
s_indices_batch, s_values_batch = sparse_documents(text_documents)

Генерация векторов...
Генерация разреженных векторов...


In [185]:
# Note the "metadata." prefix
nested_fields = ["metadata.discipline", "metadata.publisher", "metadata.grade"]

for field in nested_fields:
    client.create_payload_index(
        collection_name=collection_name,
        field_name=field,
        field_schema=models.PayloadSchemaType.KEYWORD
    )
    print(f"Index created for nested field: '{field}'")

Index created for nested field: 'metadata.discipline'
Index created for nested field: 'metadata.publisher'
Index created for nested field: 'metadata.grade'


In [167]:

point_ids = [str(uuid.uuid4()) for _ in text_documents]

# Сохраняем в Qdrant
points = [models.PointStruct(
    id=point_ids[i],
    vector={
        "voyage-dense": dense,
        "bge-sparse": models.SparseVector(indices=keys, values=vals)
    },
    payload={
        "page_content": text,
        "images": image,
        "metadata": {
            "discipline": DISCIPLINE,
            "grade": GRADE,
            "publisher": PUBLISHER
        }
    }
) for i, (text, image, dense, keys, vals) in enumerate(zip(text_documents, chunk_images_batchs, dense_vectors, s_indices_batch, s_values_batch))] 

# batch_points = []
# for point in points:
#     batch_points.append(point)
#     if len(batch_points) >= 20: # Загружаем батчами
#         client.upsert(collection_name=collection_name, points=batch_points)
#         batch_points = []

# if batch_points:
#     client.upsert(collection_name=collection_name, points=batch_points)

client.upsert(collection_name=collection_name, points=points)


print("✅ Индексация по заголовкам завершена!")

✅ Индексация по заголовкам завершена!


In [168]:
def sparse_query(query):
    output = sparse_model.encode(query,
    return_dense=False, 
    return_sparse=True, 
    return_colbert_vecs=False)
    return [int(k) for k in output["lexical_weights"].keys()], [float(v) for v in output["lexical_weights"].values()]

In [169]:

# 3. Поиск (остается таким же, но теперь возвращает целые темы)
def hybrid_search(query, k=20):
    d_vec = dense_model.embed_query(query)
    s_indices, s_values = sparse_query(query)

    return client.query_points(
        collection_name=collection_name,
        prefetch=[
            # Первый поток: Dense (Gemini)
            models.Prefetch(
                query=d_vec,
                using="voyage-dense", # Имя dense вектора
                limit=k,
            ),
            # Второй поток: Sparse (BGE-M3)
            models.Prefetch(
                query=models.SparseVector(indices=s_indices, values=s_values),
                using="bge-sparse", # Имя sparse вектора
                limit=k,
            ),
        ],
        # Объединяем результаты через RRF (Reciprocal Rank Fusion)
        # Это лучший способ для гибридного поиска
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        limit=k,
    )

docs = hybrid_search("канси деген кім?")
docs

QueryResponse(points=[ScoredPoint(id='bfeb6783-5fa0-41df-a394-c5b5d43c5ce4', version=1, score=0.7, payload={'page_content': '### 1. Қазақ хандығының XVIII ғасырдың бас кезіндегі ішкі және сыртқы жағдайы.  \n**Тірек сөздер:**\n* Жоңғар\n* Қайып хан\n* Канси\n* Сепаратизм\n* Цеван-Рабтан\n* «Елім-ай»  \n**Анықта!**\nҚазақ қоғамында сепаратизмнің күшеюіне не әсер етті?  \nТәуке хан билік еткен жылдары Қазақ хандығы орталықтандырылған мемлекет болып қала берді. Қазақ хандығында саяси тұрақтылық орнады, ішкі алауыздық азайды. Ауызбіршіліктің арқасында елдің орасан зор аумағының тұтастығы сақталды.  \nТәуке ханнан кейін оның мұрагері Қайып хан қазақ мемлекетінің бұрынғы ұлылығын берік ұстап тұра алмады. Сұлтандардың тарапынан наразылық күшейіп, елде хандық билікке талас өрши түсті. Мұның өзі дәстүрлі Қазақ хандығын дағдарыс пен күйзеліске ұшыратты. Ру-тайпа мен жүздер арасында алауыздық орын алды. Сұлтандардың аймақтық сепаратизмі күшейді. Орталық хандық биліктің әлсіреуі Кіші жүзде – Әбілқа

In [170]:
for i in docs.points:
    print(i.payload)

{'page_content': '### 1. Қазақ хандығының XVIII ғасырдың бас кезіндегі ішкі және сыртқы жағдайы.  \n**Тірек сөздер:**\n* Жоңғар\n* Қайып хан\n* Канси\n* Сепаратизм\n* Цеван-Рабтан\n* «Елім-ай»  \n**Анықта!**\nҚазақ қоғамында сепаратизмнің күшеюіне не әсер етті?  \nТәуке хан билік еткен жылдары Қазақ хандығы орталықтандырылған мемлекет болып қала берді. Қазақ хандығында саяси тұрақтылық орнады, ішкі алауыздық азайды. Ауызбіршіліктің арқасында елдің орасан зор аумағының тұтастығы сақталды.  \nТәуке ханнан кейін оның мұрагері Қайып хан қазақ мемлекетінің бұрынғы ұлылығын берік ұстап тұра алмады. Сұлтандардың тарапынан наразылық күшейіп, елде хандық билікке талас өрши түсті. Мұның өзі дәстүрлі Қазақ хандығын дағдарыс пен күйзеліске ұшыратты. Ру-тайпа мен жүздер арасында алауыздық орын алды. Сұлтандардың аймақтық сепаратизмі күшейді. Орталық хандық биліктің әлсіреуі Кіші жүзде – Әбілқайырдың, Орта жүзде – Болат, Сәмеке, Күшіктің, Ұлы жүзде Жолбарыс хандардың жағдайын нығайта түсті.  \n6  \n

In [171]:
for i in docs.points:
    print(i)

id='bfeb6783-5fa0-41df-a394-c5b5d43c5ce4' version=1 score=0.7 payload={'page_content': '### 1. Қазақ хандығының XVIII ғасырдың бас кезіндегі ішкі және сыртқы жағдайы.  \n**Тірек сөздер:**\n* Жоңғар\n* Қайып хан\n* Канси\n* Сепаратизм\n* Цеван-Рабтан\n* «Елім-ай»  \n**Анықта!**\nҚазақ қоғамында сепаратизмнің күшеюіне не әсер етті?  \nТәуке хан билік еткен жылдары Қазақ хандығы орталықтандырылған мемлекет болып қала берді. Қазақ хандығында саяси тұрақтылық орнады, ішкі алауыздық азайды. Ауызбіршіліктің арқасында елдің орасан зор аумағының тұтастығы сақталды.  \nТәуке ханнан кейін оның мұрагері Қайып хан қазақ мемлекетінің бұрынғы ұлылығын берік ұстап тұра алмады. Сұлтандардың тарапынан наразылық күшейіп, елде хандық билікке талас өрши түсті. Мұның өзі дәстүрлі Қазақ хандығын дағдарыс пен күйзеліске ұшыратты. Ру-тайпа мен жүздер арасында алауыздық орын алды. Сұлтандардың аймақтық сепаратизмі күшейді. Орталық хандық биліктің әлсіреуі Кіші жүзде – Әбілқайырдың, Орта жүзде – Болат, Сәмеке, К

In [172]:
for i in docs.points:
    print(i.payload["metadata"])
    print(i.payload["images"])

{'discipline': 'Қазақстан тарихы', 'grade': '10', 'publisher': 'Мектеп'}
[]
{'discipline': 'Қазақстан тарихы', 'grade': '10', 'publisher': 'Мектеп'}
[]
{'discipline': 'Қазақстан тарихы', 'grade': '10', 'publisher': 'Мектеп'}
[]
{'discipline': 'Қазақстан тарихы', 'grade': '10', 'publisher': 'Мектеп'}
[]
{'discipline': 'Қазақстан тарихы', 'grade': '10', 'publisher': 'Мектеп'}
[]


In [183]:
from qdrant_client import models

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

import voyageai
from langchain_voyageai import VoyageAIRerank


# Initialize Voyage client
vo = voyageai.Client(api_key=VOYAGE_API)

# --- 1. Setup Models ---
# LLM (Gemini 1.5 Flash for speed/cost)
llm = ChatGoogleGenerativeAI(model="gemini-3-flash-preview", temperature=1, api_key=GEMINI_API_KEY)

# --- 2. Custom Hybrid Retrieval Logic ---

def hybrid_retriever_func(query: str, metadata_filter: dict = None):
    """
    Custom function to perform Hybrid Search in Qdrant 
    and return content + images from the payload.
    """

    qdrant_filter = None
    if metadata_filter:
        conditions = []
        for key, value in metadata_filter.items():
            # Мы используем "metadata.key", так как в payload мы вложили словарь metadata
            conditions.append(
                models.FieldCondition(
                    key=f"metadata.{key}", 
                    match=models.MatchValue(value=value)
                )
            )
        qdrant_filter = models.Filter(must=conditions)

    # Generate Dense Vector
    query_dense = dense_model.embed_query(query)
    
    # Generate Sparse Vector
    keys, vals = sparse_query(query)
    query_sparse = models.SparseVector(
        indices=keys,
        values=vals
    )

    # Perform Hybrid Search using RRF (Reciprocal Rank Fusion)
    search_results = client.query_points(
        collection_name=collection_name,
        prefetch=[
            models.Prefetch(query=query_dense, using="voyage-dense", limit=20, filter=qdrant_filter),
            models.Prefetch(query=query_sparse, using="bge-sparse", limit=20, filter=qdrant_filter),
        ],
        query=models.FusionQuery(fusion=models.Fusion.RRF),
        limit=20,
        with_payload=True
    )
    if not search_results.points:
        return {"context_text": "Информация не найдена.", "images": []}
    

    # --- STEP 2: VOYAGE RERANK 2.5 (Precision) ---
    # Prepare candidates for the Reranker
    candidate_texts = [hit.payload['page_content'] for hit in search_results.points]

    
    # rerank-2 is the model name for Voyage's latest reranker
    rerank_results = vo.rerank(
        query=query, 
        documents=candidate_texts, 
        model="rerank-2.5", 
        top_k=3 # We only want the absolute best 3 now
    )

    # --- STEP 3: FORMAT RESULTS ---
    final_docs = []
    final_images = []


    for r in rerank_results.results:
        # Get the original index from the candidates
        idx = r.index
        hit = search_results.points[idx]
        
        # Add text and the images associated with this specific chunk
        final_docs.append(f"""
--- Context Segment (Relevance Score: {r.relevance_score:.4f}) ---
Кітап атауы: {hit.payload.get('metadata', {}).get('discipline', 'Unknown')}
Сынып: {hit.payload.get('metadata', {}).get('grade', 'Unknown')}
Баспа: {hit.payload.get('metadata', {}).get('publisher', 'Unknown')}

{hit.payload['page_content']}""")
        
        

        if "images" in hit.payload:
            final_images.extend(hit.payload["images"])
    

    return {"context_text": "\n\n".join(final_docs), "images": final_images}

# --- 3. Construct the Prompt ---

prompt_template = """
Сен Қазақстан тарихынан ҰБТ-ға (Ұлттық бірыңғай тестілеу) дайындайтын кәсіби репетиторсың. 
Сенің міндетің - тек қана берілген контекст негізінде студентке жауап беру.

Нұсқаулықтар:
1. Жауапты нақты фактілермен (жылдар, есімдер, оқиғалар) негізде.
2. Егер контекстте ақпарат болмаса, "Мәтінде бұл сұраққа жауап жоқ" деп айт.
3. Жауаптың соңында міндетті түрде пайдаланылған дереккөздерді көрсет. (Кітап атауы, Сыныбы, Баспасы, Кытап беттерінің нөмірлері)

Контекст:
{context}

Сұрақ: {question}
"""

def build_multimodal_prompt(input_dict):
    """
    Constructs the final list of messages (text + images) for Gemini.
    """
    question = input_dict["question"]
    context_data = input_dict["context_data"]
    
    # Text Part
    prompt_text = prompt_template.format(
        context=context_data["context_text"], 
        question=question
    )
    
    message_content = [{"type": "text", "text": prompt_text}]
    
    # Image Part (Base64)
    for img_b64 in context_data["images"][:3]: # Limit to 3 images for token safety
        message_content.append({
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{img_b64}"}
        })
        
    return [HumanMessage(content=message_content)]

# --- 4. The LangChain Expression Language (LCEL) Chain ---

from langchain_core.messages import HumanMessage

rag_chain_w_sources = {
        "context_data": lambda x: hybrid_retriever_func(x["question"], x.get("filter")),
        "question": lambda x: x["question"]
    }| RunnablePassthrough().assign(
    response=(
    RunnableLambda(build_multimodal_prompt)
    | llm
    | StrOutputParser()
    )
)


In [192]:
query_params = {
    "question": "Керей мен Жәнібек туралы не айтылған?",
    "filter": {
        "grade": "10",
        "discipline": "География"
    }
}

response = rag_chain_w_sources.invoke(query_params)
response

{'context_data': {'context_text': 'Информация не найдена.', 'images': []},
 'question': 'Керей мен Жәнібек туралы не айтылған?',
 'response': 'Мәтінде бұл сұраққа жауап жоқ.\n\n**Пайдаланылған дереккөздер:**\nБерілген контекст ("Информация не найдена").'}

In [193]:
print(response["response"])

Мәтінде бұл сұраққа жауап жоқ.

**Пайдаланылған дереккөздер:**
Берілген контекст ("Информация не найдена").


In [191]:
for key, val in response["context_data"].items():
    print(f"{val}:")


--- Context Segment (Relevance Score: 0.5508) ---
Кітап атауы: Қазақстан тарихы
Сынып: 10
Баспа: Мектеп

# §5-6. ҚАЗАҚ-ЖОҢҒАР ШАЙҚАСТАРЫ  
> Бүгінгі сабағымызда жоңғар шапқыншылығына қарсы тұрған хандар мен батырлардың ерлік істерімен танысамыз.  
Қазақ халқы қиын-қыстау уақытта тек өз күштеріне ғана үміт артып, біріге бастады. Бүкіл халықтың күш-жігерін жұмылдыру қазақтардың Бұланты және Аңырақай шайқастарында жеңістерге қол жеткізуіне мүмкіндік берді.  
| **Тірек сөздер:**- Бұланты шайқасы
- Аңырақай шайқасы
- «Қалмаққырылған»
- Аманат | ## 1. Ұлт-азаттық күрестің жетекшілері және оған қатысушылар.Басқыншы жауға қарсы күреске бүкіл қазақ халқы көтерілді. Ірі-ірі әскери жасақтарды Әбілмәмбет, Барақ, Сәмеке, Әбілқайыр, Жолбарыс сияқты хан-сұлтандар мен Шыңғысханның басқа да көптеген ұрпақтары басқарды. Жас сұлтан Абылайдың беделі дәл осы кезеңде көтерілді. Ол кейін қазақтың аса құрметті, беделді де айбынды ханына айналды.Азаттық күресіне идеялық басшылық жасауды қазақтың үш жүзінің ас